# Inference

## Read new client's data

In [40]:
import pandas as pd
import mlflow
import mlflow.pyfunc
from HotelCancellations_MLflow import load_data

In [41]:
new_customers = pd.read_csv("data/new_customers.csv")
new_customers.shape

(100, 36)

In [42]:
new_customers2 = load_data("data/new_customers.csv")
new_customers2 = new_customers2.drop('is_canceled', axis=1)
new_customers2

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,City Hotel,0,2016,March,13,22,0,1,2,0.0,...,D,0,No Deposit,NaN,NaN,0,Transient,99.00,0,1
1,City Hotel,21,2016,March,13,23,0,3,2,0.0,...,A,0,No Deposit,1.0,NaN,0,Transient-Party,62.00,0,1
2,City Hotel,418,2016,September,40,26,1,2,2,0.0,...,A,0,No Deposit,132.0,NaN,223,Transient-Party,107.00,0,0
3,City Hotel,58,2016,March,12,17,0,3,2,0.0,...,A,0,No Deposit,16.0,NaN,0,Transient,63.00,0,0
4,Resort Hotel,130,2017,July,28,9,2,0,1,0.0,...,E,0,No Deposit,242.0,NaN,0,Transient-Party,178.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,City Hotel,133,2017,August,31,4,0,2,2,0.0,...,A,0,No Deposit,9.0,NaN,0,Transient,135.00,0,0
96,City Hotel,18,2016,November,48,20,2,3,2,0.0,...,A,0,No Deposit,7.0,NaN,0,Transient,71.21,0,0
97,Resort Hotel,49,2017,March,10,10,0,2,2,0.0,...,A,0,No Deposit,NaN,NaN,0,Transient,0.00,0,0
98,City Hotel,93,2016,November,45,3,0,3,3,0.0,...,D,0,No Deposit,9.0,NaN,0,Transient,122.40,0,1


## Using logged model

In [43]:
# If using a separate server uncomment the following line
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [44]:
logged_model = 'runs:/63c61f22539f4e57bf6ec597540c5aa7/cancellations-model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
print(loaded_model.metadata.signature)

inputs: 
  ['hotel': string (required), 'lead_time': long (required), 'arrival_date_year': long (required), 'arrival_date_month': string (required), 'arrival_date_week_number': long (required), 'arrival_date_day_of_month': long (required), 'stays_in_weekend_nights': long (required), 'stays_in_week_nights': long (required), 'adults': long (required), 'children': double (required), 'babies': long (required), 'meal': string (required), 'country': string (required), 'market_segment': string (required), 'distribution_channel': string (required), 'is_repeated_guest': long (required), 'previous_cancellations': long (required), 'previous_bookings_not_canceled': long (required), 'reserved_room_type': string (required), 'assigned_room_type': string (required), 'booking_changes': long (required), 'deposit_type': string (required), 'agent': double (optional), 'company': double (optional), 'days_in_waiting_list': long (required), 'customer_type': string (required), 'adr': double (required), 'requir

In [45]:
loaded_model.predict(new_customers2)

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0], dtype=int64)

## Using staging / production models

In [46]:
model_name = "CancellationsModel"
alias = "champion"

# Cargar el modelo usando la URI con alias
model_uri = f"models:/{model_name}@{alias}"
model = mlflow.pyfunc.load_model(model_uri)


In [47]:
model.predict(new_customers)

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0], dtype=int64)

### Old version of mlflow by using stage

```python
model_name = "CancellationsModel"
stage = "Staging"

loaded_model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{stage}")